## Web crawler


import modules


In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait


import time
import numpy as np
import pandas as pd
import pickle

initialize args


In [4]:
service = Service()
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'cookies': 2, 'images': 2, 'javascript': 2, 
                            'plugins': 2, 'popups': 2, 'geolocation': 2, 
                            'notifications': 2, 'auto_select_certificate': 2, 'fullscreen': 2, 
                            'mouselock': 2, 'mixed_script': 2, 'media_stream': 2, 
                            'media_stream_mic': 2, 'media_stream_camera': 2, 'protocol_handlers': 2, 
                            'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2, 
                            'push_messaging': 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 
                            'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement': 2, 
                            'durable_storage': 2}}
options.add_experimental_option('prefs', prefs)
options.page_load_strategy = 'eager'

options.add_argument('--headless')  # 啟動Headless 無頭



extract publisher, creator, year from raw data


In [5]:
def process_rawData(creatorRaw, publisherRaw):
       creator = creatorRaw.removesuffix(', 著')
       split = publisherRaw.split(',')
       year = split[-1]
       year = year.split('[')[0]
       publisher = split[0].split(':')[-1].replace('\n', ' ')
       publisher = publisher.removeprefix(' ')
       
       return creator, publisher, year
       

crawler function


In [31]:
def crawl_book(bookTitle):
       driver = webdriver.Chrome(service=service, options=options)
       wait = WebDriverWait(driver, timeout=10, poll_frequency=1)
       baseUrl = 'https://metadata.ncl.edu.tw/blstkmc/blstkm#tudorkmtop'
       driver.get(baseUrl)
       
       # set search configs
       x1 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[2]/td[1]/select'
       select = Select(wait.until(
                     EC.presence_of_element_located((By.XPATH, x1))
                     ))
       select.select_by_value('n_TI')
       x2 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/select'
       select = Select(driver.find_element(By.XPATH, x2))
       select.select_by_value('C')
       # enter book name
       x3 = '//*[@id="hotname"]'
       query = driver.find_element(By.XPATH, x3)
       query.clear()
       query.send_keys(bookTitle)
       # press search
       x4 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table[1]/tbody/tr[2]/td/table/tbody/tr[5]/td/input[1]'
       searchBtn = wait.until(
                     EC.presence_of_element_located((By.XPATH, x4))
                     )
       searchBtn.click()
       
       
       try:
              # sort by year , ascending
              x6 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/select'
              select = Select( wait.until(
                     EC.presence_of_element_located((By.XPATH, x6))
              ))
              select.select_by_value('YR')
              x7 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[3]/table/tbody/tr/td[3]/select'
              select = Select( wait.until(
                     EC.presence_of_element_located((By.XPATH, x7))
              ))
              select.select_by_value('0')
              x8 = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[4]/td[1]/table/tbody/tr/td[2]/input'
              sortBtn =  wait.until(
                     EC.presence_of_element_located((By.XPATH, x8))
              )
              sortBtn.click()
              
              xCreator = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[5]/td/table/tbody/tr[2]/td[3]'
              xPublisher  = '/html/body/form/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[5]/td/table/tbody/tr[2]/td[4]'
              creatorRaw =   wait.until(
                     EC.presence_of_element_located((By.XPATH, xCreator))
              ).text
              publisherRaw =  wait.until(
                     EC.presence_of_element_located((By.XPATH, xPublisher))
              ).text
              creator, publisher, year = process_rawData(creatorRaw, publisherRaw)
       except:
              creator, publisher, year = 'error', 'error', 'error'
              print('error', bookTitle)  
       driver.close()  
       
       return pd.Series( [bookTitle, creator, publisher, year], index=['title','creator','publisher','year'])


test one book 

In [32]:
crawl_book('三體')

title           三體
creator        劉慈欣
publisher      貓頭鷹
year          2023
dtype: object

In [16]:
df = pd.read_csv('書籍.csv')
df = df.dropna().reset_index()
print(df['書名'])

0               龍鱗焰火
1               龍族秘錄
2              黑暗的左手
3               黃海童話
4        非理性時代 牛頓加農砲
           ...      
496        [簡]納米魔幻兵團
497    [簡]現代中國科幻文學主潮
498     黃禍 3/3黃禍反撲世界
499     黃禍 1/3大陸內戰爆發
500    黃禍  2/3台灣生死存亡
Name: 書名, Length: 501, dtype: object


In [17]:
errorIndex = []
start = 0
checkpoint = 50
new_df = pd.DataFrame()

In [33]:
for i in range(start, start+checkpoint):
      series = crawl_book(df['書名'][i])
      if(series['creator']== 'error'):
            errorIndex.append(i) 
      new_df = pd.concat([new_df, series],axis=1)
      if(i%10 == 0):
            if(i!=0): 
                  print()
            print(f'done{i}/{start+checkpoint}', end = '')      
      print('.', end = '')

new_df.to_csv(f'./csv/{start}-{start+checkpoint}.csv')
start+=checkpoint


error 衛斯理回憶錄7瀰散

done50/100.error 衛斯理回憶錄2同位
.error 衛斯理回憶錄1錯構
.....error 蘋果核戰 vo.1-vo.3
...
done60/100....error 艾西莫夫科普教室 (下)
.error 艾西莫夫科普教室 (上)
.error 艾西莫夫 科普教室 (中)
.error 臺灣科幻文學薪火錄(1956~2005)
...
done70/100.error 群（上／下）
..error 美洲來的哥倫布
..error 縱橫X檔案 (下)
.error 縱橫X檔案 (上)
....
done80/100.error 第五類接觸 世界科幻文學簡史
.....error 穿越時空3000年 (下)
.error 穿越時空3000年 (上)
.error 空想科學讀本6
.error 空想科學讀本4
.error 空想科學讀本3

done90/100.error 空想科學讀本2
.error 空想科學讀本2
.error 空想科學讀本1
...error 科幻電影的預言與真實
..error 科幻文學論鋼
..

In [34]:
print(errorIndex)

[20, 27, 30, 35, 36, 47, 48, 49, 50, 51, 52, 57, 64, 65, 66, 71, 73, 75, 76, 80, 81, 86, 50, 51, 52, 57, 64, 65, 66, 67, 71, 73, 75, 76, 81, 86, 87, 88, 89, 90, 91, 92, 93, 96, 98]


In [35]:

with open(f'./var/var{start-checkpoint}-{start}.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([start, checkpoint, errorIndex, new_df], f)

# Getting back the objects:


In [ ]:
with open(f'./var/var{start-checkpoint}-{start}.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    start,checkpoint,errorIndex = pickle.load(f)